[View in Colaboratory](https://colab.research.google.com/github/thayumaanavan/Generative-Adversarial-Network/blob/master/GAN.ipynb)

In [4]:
!pip install Pillow

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [0]:
from __future__ import print_function

from keras import backend as K
K.set_image_dim_ordering('th') # ensure our dimension notation matches

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, AveragePooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD, Adam
from keras.datasets import mnist
from keras import utils
import numpy as np
from PIL import Image, ImageOps
import argparse
import math

import os
import os.path

import glob

In [0]:
def generator_model():
  model = Sequential()
  model.add(Dense(input_dim=100,units=1024))
  model.add(Activation('tanh'))
  model.add(Dense(128*8*8))
  model.add(BatchNormalization())
  model.add(Activation('tanh'))
  model.add(Reshape((128,8,8), input_shape=(128*8*8,)))
  model.add(UpSampling2D(size=(4,4)))
  model.add(Conv2D(1,(5,5),padding='same'))
  model.add(Activation('tanh'))
  model.add(UpSampling2D(size=(4,4)))
  model.add(Conv2D(1,(5,5),padding='same'))
  model.add(Activation('tanh'))
  return model

In [0]:
def discriminator_model():
  model = Sequential()
  model.add(Conv2D(64,(5,5), padding='same',input_shape=(1,128,128)))
  model.add(Activation('tanh'))
  model.add(AveragePooling2D(pool_size=(4,4)))
  model.add(Conv2D(128,(5,5)))
  model.add(Activation('tanh'))
  model.add(AveragePooling2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(256))
  model.add(Activation('tanh'))
  model.add(Dense(1))
  model.add(Activation('sigmoid'))
  return model

In [0]:
def gen_disc(generator,discriminator):
  model = Sequential()
  model.add(generator)
  discriminator.trainable = False
  model.add(discriminator)
  return model

In [0]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[2:]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[0, :, :]
    return image

In [12]:
model = generator_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1024)              103424    
_________________________________________________________________
activation_4 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 8192)              8396800   
_________________________________________________________________
batch_normalization_2 (Batch (None, 8192)              32768     
_________________________________________________________________
activation_5 (Activation)    (None, 8192)              0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 128, 8, 8)         0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 128, 32, 32)       0         
__________

In [0]:
def load_data(pixels=128, verbose=False):
    print("Loading data")
    X_train = []
    paths = glob.glob(os.path.normpath(os.getcwd() + '/pictures/*.JPG'))
    for path in paths:
        if verbose: print(path)
        im = Image.open(path)
        im = ImageOps.fit(im, (pixels, pixels), Image.ANTIALIAS)
        im = ImageOps.grayscale(im)
        #im.show()
        im = np.asarray(im)
        X_train.append(im)
    print("Finished loading data")
    return np.array(X_train)

In [15]:
paths = glob.glob(os.path.normpath(os.getcwd() + '/pictures/*.jpg'))
print(paths)
print(glob.glob(os.path.normpath(os.getcwd())))

['C:\\Users\\chitrara\\pictures\\Aadhar.JPG', 'C:\\Users\\chitrara\\pictures\\AADHAR_BACK.jpg', 'C:\\Users\\chitrara\\pictures\\AADHAR_FRONT.jpg', 'C:\\Users\\chitrara\\pictures\\IMG_0114.JPG', 'C:\\Users\\chitrara\\pictures\\IMG_0746.JPG', 'C:\\Users\\chitrara\\pictures\\IMG_0899.JPG', 'C:\\Users\\chitrara\\pictures\\IMG_0900.JPG', 'C:\\Users\\chitrara\\pictures\\IMG_0901.JPG', 'C:\\Users\\chitrara\\pictures\\IMG_0902.JPG', 'C:\\Users\\chitrara\\pictures\\IMG_0903.JPG', 'C:\\Users\\chitrara\\pictures\\license.jpg', 'C:\\Users\\chitrara\\pictures\\pan-front.jpg', 'C:\\Users\\chitrara\\pictures\\PAN_CARD.JPG', 'C:\\Users\\chitrara\\pictures\\photo.jpg', 'C:\\Users\\chitrara\\pictures\\Tin.jpg']
['C:\\Users\\chitrara']


In [0]:
def train(epochs, BATCH_SIZE, weights=False):
  X_train = load_data()
  X_train = (X_train.astype(np.float32)- 127.5)/127.5
  X_train = X_train.reshape((X_train.shape[0], 1) + X_train.shape[1:])
  
  discriminator = discriminator_model()
  generator = generator_model()
  
  if weights:
    generator.load_weights('generator.h5')
    discriminator.load_weights('discriminator.h5')
  disc_on_gen = gen_disc(generator,discriminator)
    
  d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
  g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
   
  generator.compile(loss='binary_crossentropy', optimizer="SGD")
  disc_on_gen.compile(loss='binary_crossentropy', optimizer = g_optim)
  discriminator.trainable = True
  discriminator.compile(loss='binary_crossentropy', optimizer= d_optim)
  noise = np.zeros((BATCH_SIZE, 100))
    
  for epoch in range(epochs):
    print("Epoch:", epoch)
    no_of_batches = int(X_train.shape[0]/BATCH_SIZE)
    print("No of Batches:", no_of_batches)
    
    for index in range(no_of_batches):
      for i in range(BATCH_SIZE):
        noise[i, :] = np.random.uniform(-1, 1, 100)
      image_batch = X_train[index*BATCH_SIZE: (index+1)*BATCH_SIZE]
      generated_images = generator.predict(noise, verbose=0)
       
      if index%20==0 and epoch%10==0:
        image = combine_images(generated_images)
        image = image*127.5+127.5
        destpath = os.path.normpath(os.getcwd()+"/generated_images/"+str(epoch)+"_"+str(index)+".png")
        Image.fromarray(image.astype(np.uint8)).save(destpath)
        
      X = np.concatenate((image_batch, generated_images))
      y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
      d_loss = discriminator.train_on_batch(X, y)
      print("batch %d d_loss : %f" % (index, d_loss))
        
      for i in range(BATCH_SIZE):
        noise[i, :] = np.random.uniform(-1, 1, 100)
          
      discriminator.trainable = False
      g_loss = disc_on_gen.train_on_batch(noise, [1]* BATCH_SIZE)
      discriminator.trainable = True
      print("batch %d g_loss : %f" % (index, g_loss))
      if epoch % 10 == 9:
        generator.save_weights('generator.h5', True)
        discriminator.save_weights('discriminator.h5', True)      

In [0]:
def clean(image):
    for i in range(1, image.shape[0] - 1):
        for j in range(1, image.shape[1] - 1):
            if image[i][j] + image[i+1][j] + image[i][j+1] + image[i-1][j] + image[i][j-1] > 127 * 5:
                image[i][j] = 255
    return image
def generate(BATCH_SIZE):
    generator = generator_model()
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    generator.load_weights('generator.h5')
    noise = np.zeros((BATCH_SIZE, 100))
    a = np.random.uniform(-1, 1, 100)
    b = np.random.uniform(-1, 1, 100)
    grad = (b - a) / BATCH_SIZE
    for i in range(BATCH_SIZE):
        noise[i, :] = np.random.uniform(-1, 1, 100)
    generated_images = generator.predict(noise, verbose=1)
    #image = combine_images(generated_images)
    print(generated_images.shape)
    for image in generated_images:
        image = image[0]
        image = image*127.5+127.5
        Image.fromarray(image.astype(np.uint8)).save("dirty.png")
        Image.fromarray(image.astype(np.uint8)).show()
        clean(image)
        image = Image.fromarray(image.astype(np.uint8))
        image.show()        
        image.save("clean.png")


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", type=str)
    parser.add_argument("--batch_size", type=int, default=128)
    parser.add_argument("--nice", dest="nice", action="store_true")
    parser.set_defaults(nice=False)
    args = parser.parse_args()
    return args

In [25]:
train(400,10,False)

Loading data
Finished loading data
Epoch: 0
No of Batches: 1
batch 0 d_loss : 0.704457
batch 0 g_loss : 0.703132
Epoch: 1
No of Batches: 1
batch 0 d_loss : 0.698280
batch 0 g_loss : 0.684526
Epoch: 2
No of Batches: 1
batch 0 d_loss : 0.685302
batch 0 g_loss : 0.702063
Epoch: 3
No of Batches: 1
batch 0 d_loss : 0.649401
batch 0 g_loss : 0.712410
Epoch: 4
No of Batches: 1
batch 0 d_loss : 0.668621
batch 0 g_loss : 0.692460
Epoch: 5
No of Batches: 1
batch 0 d_loss : 0.627873
batch 0 g_loss : 0.690354
Epoch: 6
No of Batches: 1
batch 0 d_loss : 0.616889
batch 0 g_loss : 0.712667
Epoch: 7
No of Batches: 1
batch 0 d_loss : 0.579413
batch 0 g_loss : 0.687022
Epoch: 8
No of Batches: 1
batch 0 d_loss : 0.547028
batch 0 g_loss : 0.706439
Epoch: 9
No of Batches: 1
batch 0 d_loss : 0.532388
batch 0 g_loss : 0.711883
Epoch: 10
No of Batches: 1
batch 0 d_loss : 0.543701
batch 0 g_loss : 0.697273
Epoch: 11
No of Batches: 1
batch 0 d_loss : 0.549333
batch 0 g_loss : 0.724353
Epoch: 12
No of Batches: 1


batch 0 g_loss : 0.505217
Epoch: 34
No of Batches: 1
batch 0 d_loss : 0.564412
batch 0 g_loss : 0.481726
Epoch: 35
No of Batches: 1
batch 0 d_loss : 0.691601
batch 0 g_loss : 0.519685
Epoch: 36
No of Batches: 1
batch 0 d_loss : 0.481891
batch 0 g_loss : 0.501030
Epoch: 37
No of Batches: 1
batch 0 d_loss : 0.639350
batch 0 g_loss : 0.502731
Epoch: 38
No of Batches: 1
batch 0 d_loss : 0.587965
batch 0 g_loss : 0.505067
Epoch: 39
No of Batches: 1
batch 0 d_loss : 0.615441
batch 0 g_loss : 0.481593
Epoch: 40
No of Batches: 1
batch 0 d_loss : 0.602917
batch 0 g_loss : 0.504287
Epoch: 41
No of Batches: 1
batch 0 d_loss : 0.657570
batch 0 g_loss : 0.491738
Epoch: 42
No of Batches: 1
batch 0 d_loss : 0.578981
batch 0 g_loss : 0.537179
Epoch: 43
No of Batches: 1
batch 0 d_loss : 0.626708
batch 0 g_loss : 0.561132
Epoch: 44
No of Batches: 1
batch 0 d_loss : 0.643364
batch 0 g_loss : 0.594384
Epoch: 45
No of Batches: 1
batch 0 d_loss : 0.618909
batch 0 g_loss : 0.566099
Epoch: 46
No of Batches: 1

batch 0 d_loss : 0.577903
batch 0 g_loss : 0.971121
Epoch: 69
No of Batches: 1
batch 0 d_loss : 0.548741
batch 0 g_loss : 0.945508
Epoch: 70
No of Batches: 1
batch 0 d_loss : 0.493442
batch 0 g_loss : 1.025162
Epoch: 71
No of Batches: 1
batch 0 d_loss : 0.539806
batch 0 g_loss : 1.010593
Epoch: 72
No of Batches: 1
batch 0 d_loss : 0.496005
batch 0 g_loss : 1.024640
Epoch: 73
No of Batches: 1
batch 0 d_loss : 0.518181
batch 0 g_loss : 0.999069
Epoch: 74
No of Batches: 1
batch 0 d_loss : 0.477514
batch 0 g_loss : 1.028671
Epoch: 75
No of Batches: 1
batch 0 d_loss : 0.521812
batch 0 g_loss : 1.026817
Epoch: 76
No of Batches: 1
batch 0 d_loss : 0.490385
batch 0 g_loss : 1.027414
Epoch: 77
No of Batches: 1
batch 0 d_loss : 0.493028
batch 0 g_loss : 1.024083
Epoch: 78
No of Batches: 1
batch 0 d_loss : 0.499528
batch 0 g_loss : 1.033363
Epoch: 79
No of Batches: 1
batch 0 d_loss : 0.465511
batch 0 g_loss : 1.101747
Epoch: 80
No of Batches: 1
batch 0 d_loss : 0.452191
batch 0 g_loss : 1.062659


batch 0 d_loss : 0.446575
batch 0 g_loss : 1.135550
Epoch: 103
No of Batches: 1
batch 0 d_loss : 0.426775
batch 0 g_loss : 1.060943
Epoch: 104
No of Batches: 1
batch 0 d_loss : 0.457324
batch 0 g_loss : 1.300130
Epoch: 105
No of Batches: 1
batch 0 d_loss : 0.460820
batch 0 g_loss : 1.004561
Epoch: 106
No of Batches: 1
batch 0 d_loss : 0.449057
batch 0 g_loss : 1.144622
Epoch: 107
No of Batches: 1
batch 0 d_loss : 0.410607
batch 0 g_loss : 1.000196
Epoch: 108
No of Batches: 1
batch 0 d_loss : 0.410911
batch 0 g_loss : 1.032206
Epoch: 109
No of Batches: 1
batch 0 d_loss : 0.471953
batch 0 g_loss : 0.938236
Epoch: 110
No of Batches: 1
batch 0 d_loss : 0.453368
batch 0 g_loss : 1.064307
Epoch: 111
No of Batches: 1
batch 0 d_loss : 0.438616
batch 0 g_loss : 0.988439
Epoch: 112
No of Batches: 1
batch 0 d_loss : 0.442487
batch 0 g_loss : 1.116153
Epoch: 113
No of Batches: 1
batch 0 d_loss : 0.475618
batch 0 g_loss : 1.104376
Epoch: 114
No of Batches: 1
batch 0 d_loss : 0.489238
batch 0 g_loss

batch 0 d_loss : 0.398613
batch 0 g_loss : 0.887394
Epoch: 137
No of Batches: 1
batch 0 d_loss : 0.497581
batch 0 g_loss : 0.944958
Epoch: 138
No of Batches: 1
batch 0 d_loss : 0.447999
batch 0 g_loss : 0.818649
Epoch: 139
No of Batches: 1
batch 0 d_loss : 0.506088
batch 0 g_loss : 0.857293
Epoch: 140
No of Batches: 1
batch 0 d_loss : 0.493108
batch 0 g_loss : 0.976782
Epoch: 141
No of Batches: 1
batch 0 d_loss : 0.495635
batch 0 g_loss : 0.917983
Epoch: 142
No of Batches: 1
batch 0 d_loss : 0.510783
batch 0 g_loss : 0.754834
Epoch: 143
No of Batches: 1
batch 0 d_loss : 0.399727
batch 0 g_loss : 1.103952
Epoch: 144
No of Batches: 1
batch 0 d_loss : 0.452227
batch 0 g_loss : 1.191593
Epoch: 145
No of Batches: 1
batch 0 d_loss : 0.453082
batch 0 g_loss : 0.968089
Epoch: 146
No of Batches: 1
batch 0 d_loss : 0.509339
batch 0 g_loss : 0.982546
Epoch: 147
No of Batches: 1
batch 0 d_loss : 0.396063
batch 0 g_loss : 0.896366
Epoch: 148
No of Batches: 1
batch 0 d_loss : 0.396229
batch 0 g_loss

...

batch 0 g_loss : 1.025702
Epoch: 239
No of Batches: 1
batch 0 d_loss : 0.472355
batch 0 g_loss : 0.972162
Epoch: 240
No of Batches: 1
batch 0 d_loss : 0.470836
batch 0 g_loss : 1.089755
Epoch: 241
No of Batches: 1
batch 0 d_loss : 0.481125
batch 0 g_loss : 1.263684
Epoch: 242
No of Batches: 1
batch 0 d_loss : 0.423052
batch 0 g_loss : 0.933123
Epoch: 243
No of Batches: 1
batch 0 d_loss : 0.592404
batch 0 g_loss : 0.871934
Epoch: 244
No of Batches: 1
batch 0 d_loss : 0.420779
batch 0 g_loss : 0.845110
Epoch: 245
No of Batches: 1
batch 0 d_loss : 0.499756
batch 0 g_loss : 1.002224
Epoch: 246
No of Batches: 1
batch 0 d_loss : 0.454215
batch 0 g_loss : 1.130938
Epoch: 247
No of Batches: 1
batch 0 d_loss : 0.407627
batch 0 g_loss : 1.055792
Epoch: 248
No of Batches: 1
batch 0 d_loss : 0.449658
batch 0 g_loss : 0.886515
Epoch: 249
No of Batches: 1
batch 0 d_loss : 0.483169
batch 0 g_loss : 1.173098
Epoch: 250
No of Batches: 1
batch 0 d_loss : 0.382386
batch 0 g_loss : 0.962114
Epoch: 251
No 

batch 0 g_loss : 1.065543
Epoch: 273
No of Batches: 1
batch 0 d_loss : 0.413629
batch 0 g_loss : 1.088809
Epoch: 274
No of Batches: 1
batch 0 d_loss : 0.489296
batch 0 g_loss : 1.206415
Epoch: 275
No of Batches: 1
batch 0 d_loss : 0.468371
batch 0 g_loss : 1.298251
Epoch: 276
No of Batches: 1
batch 0 d_loss : 0.458759
batch 0 g_loss : 0.980554
Epoch: 277
No of Batches: 1
batch 0 d_loss : 0.543403
batch 0 g_loss : 0.904754
Epoch: 278
No of Batches: 1
batch 0 d_loss : 0.518126
batch 0 g_loss : 1.119904
Epoch: 279
No of Batches: 1
batch 0 d_loss : 0.657592
batch 0 g_loss : 0.945556
Epoch: 280
No of Batches: 1
batch 0 d_loss : 0.450677
batch 0 g_loss : 0.845055
Epoch: 281
No of Batches: 1
batch 0 d_loss : 0.510063
batch 0 g_loss : 0.846726
Epoch: 282
No of Batches: 1
batch 0 d_loss : 0.509884
batch 0 g_loss : 0.826927
Epoch: 283
No of Batches: 1
batch 0 d_loss : 0.576073
batch 0 g_loss : 1.088739
Epoch: 284
No of Batches: 1
batch 0 d_loss : 0.460389
batch 0 g_loss : 1.023988
Epoch: 285
No 

batch 0 g_loss : 1.035018
Epoch: 307
No of Batches: 1
batch 0 d_loss : 0.366367
batch 0 g_loss : 1.235767
Epoch: 308
No of Batches: 1
batch 0 d_loss : 0.485797
batch 0 g_loss : 1.290558
Epoch: 309
No of Batches: 1
batch 0 d_loss : 0.398656
batch 0 g_loss : 1.038404
Epoch: 310
No of Batches: 1
batch 0 d_loss : 0.451283
batch 0 g_loss : 0.950195
Epoch: 311
No of Batches: 1
batch 0 d_loss : 0.399878
batch 0 g_loss : 0.808119
Epoch: 312
No of Batches: 1
batch 0 d_loss : 0.430522
batch 0 g_loss : 1.018111
Epoch: 313
No of Batches: 1
batch 0 d_loss : 0.449204
batch 0 g_loss : 1.171330
Epoch: 314
No of Batches: 1
batch 0 d_loss : 0.410009
batch 0 g_loss : 1.160059
Epoch: 315
No of Batches: 1
batch 0 d_loss : 0.495374
batch 0 g_loss : 0.863662
Epoch: 316
No of Batches: 1
batch 0 d_loss : 0.442500
batch 0 g_loss : 0.945446
Epoch: 317
No of Batches: 1
batch 0 d_loss : 0.468589
batch 0 g_loss : 1.173623
Epoch: 318
No of Batches: 1
batch 0 d_loss : 0.380861
batch 0 g_loss : 1.222677
Epoch: 319
No 

batch 0 d_loss : 0.381998
batch 0 g_loss : 1.338053
Epoch: 341
No of Batches: 1
batch 0 d_loss : 0.447514
batch 0 g_loss : 1.226182
Epoch: 342
No of Batches: 1
batch 0 d_loss : 0.389588
batch 0 g_loss : 1.134127
Epoch: 343
No of Batches: 1
batch 0 d_loss : 0.440209
batch 0 g_loss : 1.045335
Epoch: 344
No of Batches: 1
batch 0 d_loss : 0.481026
batch 0 g_loss : 1.553114
Epoch: 345
No of Batches: 1
batch 0 d_loss : 0.577713
batch 0 g_loss : 1.262116
Epoch: 346
No of Batches: 1
batch 0 d_loss : 0.345282
batch 0 g_loss : 1.261297
Epoch: 347
No of Batches: 1
batch 0 d_loss : 0.406795
batch 0 g_loss : 1.543952
Epoch: 348
No of Batches: 1
batch 0 d_loss : 0.407208
batch 0 g_loss : 1.484653
Epoch: 349
No of Batches: 1
batch 0 d_loss : 0.371934
batch 0 g_loss : 1.478122
Epoch: 350
No of Batches: 1
batch 0 d_loss : 0.575845
batch 0 g_loss : 1.383080
Epoch: 351
No of Batches: 1
batch 0 d_loss : 0.658394
batch 0 g_loss : 1.262179
Epoch: 352
No of Batches: 1
batch 0 d_loss : 0.536917
batch 0 g_loss

batch 0 d_loss : 0.425110
batch 0 g_loss : 1.835354
Epoch: 375
No of Batches: 1
batch 0 d_loss : 0.434260
batch 0 g_loss : 1.004231
Epoch: 376
No of Batches: 1
batch 0 d_loss : 0.436195
batch 0 g_loss : 1.353432
Epoch: 377
No of Batches: 1
batch 0 d_loss : 0.412208
batch 0 g_loss : 1.156767
Epoch: 378
No of Batches: 1
batch 0 d_loss : 0.414683
batch 0 g_loss : 1.314322
Epoch: 379
No of Batches: 1
batch 0 d_loss : 0.397227
batch 0 g_loss : 1.538745
Epoch: 380
No of Batches: 1
batch 0 d_loss : 0.343011
batch 0 g_loss : 1.546887
Epoch: 381
No of Batches: 1
batch 0 d_loss : 0.464414
batch 0 g_loss : 1.414082
Epoch: 382
No of Batches: 1
batch 0 d_loss : 0.412650
batch 0 g_loss : 1.723842
Epoch: 383
No of Batches: 1
batch 0 d_loss : 0.426996
batch 0 g_loss : 1.530404
Epoch: 384
No of Batches: 1
batch 0 d_loss : 0.434956
batch 0 g_loss : 1.210563
Epoch: 385
No of Batches: 1
batch 0 d_loss : 0.405110
batch 0 g_loss : 1.547468
Epoch: 386
No of Batches: 1
batch 0 d_loss : 0.452834
batch 0 g_loss

In [27]:
generate(1)

2/2 [==============================] - 0s 145ms/step
(2, 1, 128, 128)
